In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import *

metadata = MetaData()


data_dir = '/Local/Users/bhima/temp/fda-case-study/Data/CSV'
notebook_dir = '/Local/Users/bhima/temp/fda-case-study/Data/'
report_DIR = '/Local/Users/bhima/temp/fda-case-study/Report/'

In [ ]:
connection = psycopg2.connect(
    host="localhost",
    database="openfda",
    user="bhima",
)
connection.autocommit = True
cursor = connection.cursor()

In [ ]:
sqlalchemy.

In [ ]:
!ls ../Data/CSV/*drug.csv

In [ ]:
df = pd.read_csv("../Data/CSV/2004q1_drug-event-0001-of-0001.json.drug.csv")

In [ ]:
col_info = dict(df.dtypes)
for k in col_info:
    
    if col_info[k] == 'object':
        print(k.upper(), 'CHAR')
    else:
        col_type = str(col_info[k]).upper()
        print(k.upper(), col_type)
    

In [ ]:
x = df.columns.to_list()

In [ ]:
x

In [ ]:
connection

In [ ]:
help(psycopg2)

In [ ]:
from pandas_to_postgres import (
    DataFrameCopy,
    hdf_to_postgres,
)


In [ ]:
import sqlalchemy.db

In [ ]:
help(db)

In [ ]:
from pandas_to_postgres import (
    DataFrameCopy,
    hdf_to_postgres,
)

table_model = metadata.tables['drugs']

# already loaded DataFrame & SQLAlchemy Table model
#with db.engine.connect() as c:
DataFrameCopy(df, conn=connection, table_obj=table_model).copy()

# HDF from file
#hdf_to_postgres('./data.h5', engine_args=["psycopg://..."])

# Parallel HDF from file
#hdf_to_postgres('./data.h5', engine_args=["psycopg://..."], processes=4)

In [5]:
from sqlalchemy import create_engine, MetaData
import sqlalchemy
import psycopg2 as pg

address = 'postgresql://bhima:@localhost:5432/openfda'
engine = create_engine(address)
connection = engine.raw_connection()
cursor = connection.cursor()

In [6]:
engine.

In [ ]:
from sqlalchemy import create_engine
import psycopg2 as pg
#load python script that batch loads pandas df to sql
import cStringIO

address = 'postgresql://<username>:<pswd>@<host>:<port>/<database>'
engine = create_engine(address)
connection = engine.raw_connection()
cursor = connection.cursor()

#df is the dataframe containing an index and the columns "Event" and "Day"
#create Index column to use as primary key
df.reset_index(inplace=True)
df.rename(columns={'index':'Index'}, inplace =True)

#create the table but first drop if it already exists
command = '''DROP TABLE IF EXISTS localytics_app2;
CREATE TABLE localytics_app2
(
"Index" serial primary key,
"Event" text,
"Day" timestamp without time zone,
);'''
cursor.execute(command)
connection.commit()

#stream the data using 'to_csv' and StringIO(); then use sql's 'copy_from' function
output = cStringIO.StringIO()
#ignore the index
df.to_csv(output, sep='\t', header=False, index=False)
#jump to start of stream
output.seek(0)
contents = output.getvalue()
cur = connection.cursor()
#null values become ''
cur.copy_from(output, 'localytics_app2', null="")    
connection.commit()
cur.close()